In [11]:
import pandas as pd
import geopandas
from shapely.geometry import shape, Point
import pandas as pd
from geopandas.tools import sjoin
import xarray as xr
import numpy as np

ds = xr.open_dataset('PM25/GWRwSPEC_PM25_EU_199801_199812-RH35.nc')
df = ds.to_dataframe()
df = df.reset_index()
df = df[df.PM25 > 0]
df = df.head(1000)

nuts = geopandas.read_file('nuts/NUTS_RG_20M_2021_3035.shp')
nuts = nuts.set_crs("EPSG:3035").to_crs("WGS84")

In [25]:
point=Point(-14.994999885559082, 66.34500122070312)
sum(nuts["geometry"].covers(point))
sum(nuts["geometry"].within(point))



/var/folders/8g/rym67m5d29dcjm2td47sgb_m0000gn/T/ipykernel_27843/1825436319.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  sum(nuts["geometry"].distance(point))


65039.873334505224

In [38]:
THRESHOLD=5

In [44]:
def findPointOnNUTS(nuts, lon:float, lat:float):
    nuts_inside=nuts.copy()
    point = Point(lon, lat)
    #print(point)
    # Filter level code NUTS III
    nuts_inside = nuts_inside[nuts_inside.LEVL_CODE == 3]#[r.CNTR_CODE == 'PT']
    #print(nuts_inside)
    # Filter polygons containing the point
    nuts_inside['contains'] = nuts_inside["geometry"].contains(point)
    nuts_val = nuts_inside[nuts_inside['contains']]
    #print(nuts_val)
    if len(nuts_val)==0:
        print("Not one polygon contains the point, checking distance...")
        min_distance= nuts_inside["geometry"].distance(point).min()
        argx_min=nuts_inside["geometry"].distance(point).idxmin()
        if min_distance<THRESHOLD:
            return nuts_inside.loc[argx_min].values[0]
        else:
            return None
    else:
        return nuts_val.values[0] 

#print(findPointOnNUTS(-9.255000114440918, 52.345001220703125))

def coordinates(row,nuts_file=nuts):

    result = findPointOnNUTS(nuts,row["LON"].astype('float32'), row["LAT"].astype('float32'))
    
    #print( row["LON"], row["LAT"], result)

    if(result != None):
        return result[0]
    else:
        return None

#df["COUNTRY"]=
df.iloc[1:3,:].apply(lambda x: coordinates(x), axis=1)



Not one polygon contains the point, checking distance...
Not one polygon contains the point, checking distance...


/var/folders/8g/rym67m5d29dcjm2td47sgb_m0000gn/T/ipykernel_27843/180472959.py:14: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  min_distance= nuts_inside["geometry"].distance(point).min()
/var/folders/8g/rym67m5d29dcjm2td47sgb_m0000gn/T/ipykernel_27843/180472959.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  argx_min=nuts_inside["geometry"].distance(point).idxmin()
/var/folders/8g/rym67m5d29dcjm2td47sgb_m0000gn/T/ipykernel_27843/180472959.py:14: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  min_distance= nuts_inside["geometry"].distance(point).min()
/var/folders/8g

65    I
66    I
dtype: object